# yahoo画像検索でキーワード検索&好きか嫌いかの教師データを与える
---
### 利用バージョン
python 3

## 概要
ここでは
1. SEARCH_LISTに検索キーワードを入れる。
1. 各キーワードが好き(1)か嫌い(0)か回答します。
1. それぞれのキーワードで画像検索し画像ファイルをダウンロードします。また各画像が好き(1)か嫌い(0)かのリストを作成します。　　

といった事をします。

yahoo画像検索で画像ダウンロードする機能は、[ここ](http://karaage.hatenadiary.jp/entry/2017/08/23/073000)のものを利用させてもらっています。


## 変数定義
* yahoo画像検索をスクレイピングする際に使用するメールアドレスをMY_EMAIL_ADDRに記載します。
* SEARCH_LISTに検索するキーワードを入力します。

In [1]:
MY_EMAIL_ADDR = 'mori5602@gmail.com'
SEARCH_LIST = [u'猫', u'犬']

## 教師データ設定
* 以下のセルを実行すると、各キーワードが好きかどうか聞かれるので答えます。
* デモではシンプルに、犬が好き(1)、猫が嫌い(0)で説明します。

In [2]:
CHOICE_LIST = []

for i in SEARCH_LIST:
    loopFlag = True
    while loopFlag == True:
        choice = input(u'do you like {} ?[y/N]'.format(i)).lower()

        if choice in ['y', 'ye', 'yes']:
            CHOICE_LIST.append(1)
            loopFlag = False
        elif choice in ['n', 'no']:
            CHOICE_LIST.append(0)
            loopFlag = False
    
print (CHOICE_LIST)

do you like 猫 ?[y/N]n
do you like 犬 ?[y/N]y
[0, 1]


## 画像ダウンロード＆リスト作成
以下のセルを実行すると、次の２つの結果が得られます。
* dataset/01_downloadPictに画像がダウンロードされます。
* dataset/01_download_pict_anser.csvに、ダウンロードした画像ファイル名と好き(1)or嫌い(0)の一覧が出力されます。

In [3]:
# coding: utf-8

import os
import sys
import traceback
from mimetypes import guess_extension
from time import time, sleep
from urllib.request import urlopen, Request
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

class Fetcher:
    def __init__(self, ua=''):
        self.ua = ua

    def fetch(self, url):
        req = Request(url, headers={'User-Agent': self.ua})
        try:
            with urlopen(req, timeout=3) as p:
                b_content = p.read()
                mime = p.getheader('Content-Type')
        except:
            sys.stderr.write('Error in fetching {}\n'.format(url))
            sys.stderr.write(traceback.format_exc())
            return None, None
        return b_content, mime

fetcher = Fetcher(MY_EMAIL_ADDR)

def remove_dir_and_file(top):
    for root, dirs, files in os.walk(top, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))

def fetch_and_save_img(key, word, anser):
    df = pd.DataFrame(columns=['pictFileName', 'anser'])
    
    data_dir = os.path.join(u'dataset', u'01_downloadPict')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    for i, img_url in enumerate(img_url_list(word)):
        sleep(0.1)
        img, mime = fetcher.fetch(img_url)
        if not mime or not img:
            continue
        ext = guess_extension(mime.split(';')[0])
        if ext in ('.jpe', '.jpeg'):
            ext = '.jpg'
        if not ext:
            continue
            
        pictFileName = u'{0}_{1}{2}'.format(key, i, ext)
        result_file = os.path.join(data_dir, pictFileName)

        df_tmp = pd.Series([os.path.join(u'dataset', u'01_downloadPict', pictFileName), anser], index=['pictFileName', 'anser'])
        df = df.append(df_tmp, ignore_index=True)
        
        with open(result_file, mode='wb') as f:
            f.write(img)
        print('fetched', img_url)

    return df, data_dir

def img_url_list(word):
    """
    using yahoo (this script can't use at google)
    """
    url = 'http://image.search.yahoo.co.jp/search?n=60&p={}&search.x=1'.format(quote(word))
    byte_content, _ = fetcher.fetch(url)
    structured_page = BeautifulSoup(byte_content.decode('UTF-8'), 'html.parser')
    img_link_elems = structured_page.find_all('a', attrs={'target': 'imagewin'})
    img_urls = [e.get('href') for e in img_link_elems if e.get('href').startswith('http')]
    img_urls = list(set(img_urls))
    return img_urls

if __name__ == '__main__':
    outfilePath = os.path.join(u'dataset', u'01_downloadPict')
    remove_dir_and_file(outfilePath)
                        
    df = pd.DataFrame(columns=['pictFileName', 'anser'])
    for key, (word, anser) in enumerate(zip(SEARCH_LIST, CHOICE_LIST)):
        df_tmp, data_dir = fetch_and_save_img(key, word, anser)
        df = df.append(df_tmp, ignore_index=True)
    
    # リストをランダムソートしてcsvに出力
    df = df.reindex(np.random.permutation(df.index))
    df.to_csv(os.path.join(u'dataset', '01_download_pict_anser.csv'), index=None, header=None, sep= ' ')


fetched http://3.bp.blogspot.com/-qntoBxY__og/UobsYPSQkoI/AAAAAAAA6GY/XNQ_jAvBPyo/s1600/IMG_5819.JPG
fetched http://farm1.static.flickr.com/159/334902141_175f62ff96.jpg
fetched http://i.gzn.jp/img/2013/12/08/cat-island-aoshima/P1640287_m.jpg
fetched http://farm2.static.flickr.com/1422/1103730141_dc9e11ab01.jpg
fetched http://farm4.static.flickr.com/3250/2855096963_50252e2994.jpg
fetched http://farm4.static.flickr.com/3025/2932466131_020336fbc4.jpg
fetched http://image.space.rakuten.co.jp/lg01/41/0000084841/45/imge33230cfzik2zj.jpeg
fetched http://stat.ameba.jp/user_images/4f/a6/10073854375.jpg
fetched http://farm1.static.flickr.com/18/69375046_d06dc77299.jpg
fetched http://nekopedia.jp/wp-content/uploads/2017/01/N879_kuchigahenojinoneko_TP_V.jpg
fetched http://livedoor.blogimg.jp/loveai0221/imgs/c/c/ccfbbd61.jpg
fetched http://file.digicat.blog.shinobi.jp/po07.jpg
fetched http://farm2.static.flickr.com/1372/1116259874_ef1a8d8deb.jpg
fetched https://grapee.jp/wp-content/uploads/32187_ma

fetched http://www.green-dog.com/cocokara/wp/wp-content/uploads/7564-00028-1.jpg
fetched http://illustrain.com/img/work/2016/illustrain06-inu07.png
fetched http://livedoor.blogimg.jp/karapaia_zaeega/imgs/f/e/fe9382ed.jpg
fetched http://image.blog.livedoor.jp/petshopwith/imgs/4/c/4c8d82b0.png
fetched http://farm2.static.flickr.com/1092/526120862_cdbf88506e.jpg
fetched http://livedoor.blogimg.jp/laba_q/imgs/6/c/6cc2de5b.jpg
fetched http://farm1.static.flickr.com/94/233317563_bb2478c9ff.jpg
fetched https://d3imh5q5dnm5ub.cloudfront.net/press_images/000/110/886/84e1e6f9bd947dda0e222e58e7c08defd362f7ec.jpg?1496064367
fetched https://static.pepy.jp/wp-content/uploads/2015/04/06154320/ea89e5531d8cbbabbc2fd051f8159a64-e1483583273184.jpg
fetched https://www.min-inuzukan.com/images/dog_img_long-chihuahua.jpg
fetched http://www.min-petlife.com/data/article/10338/main_10338_629bf_detail.jpg
fetched http://files.hangame.co.jp/blog/2008/52/4649b873/12/18/18634635/4649b873_1229611542575.jpg
fetched h